This notebook covers using metrics to analyze the 'accuracy' of prophet models. In this notebook, we will extend the previous example (http://pythondata.com/forecasting-time-series-data-prophet-part-3/).

### Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

%matplotlib inline
 
plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

ModuleNotFoundError: No module named 'sklearn'

### Read in the data

Read the data in from the retail sales CSV file in the examples folder then set the index to the 'date' column. We are also parsing dates in the data file.

In [ ]:
sales_df = pd.read_csv('data/retail_sales.csv', index_col='date', parse_dates=True)

In [ ]:
sales_df.head()

### Prepare for Prophet

As explained in previous prophet posts, for prophet to work, we need to change the names of these columns to 'ds' and 'y'.

In [ ]:
df = sales_df.reset_index()

In [ ]:
df.head()

Let's rename the columns as required by fbprophet. Additioinally, fbprophet doesn't like the index to be a datetime...it wants to see 'ds' as a non-index column, so we won't set an index differnetly than the integer index.

In [ ]:
df=df.rename(columns={'date':'ds', 'sales':'y'})

In [ ]:
df.head()

Now's a good time to take a look at your data.  Plot the data using pandas' ```plot``` function

In [ ]:
df.set_index('ds').y.plot()

### Running Prophet

Now, let's set prophet up to begin modeling our data using our ```promotions``` dataframe as part of the forecast

Note: Since we are using monthly data, you'll see a message from Prophet saying ```Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.```  This is OK since we are workign with monthly data but you can disable it by using ```weekly_seasonality=True``` in the instantiation of Prophet.

In [ ]:
model = Prophet(weekly_seasonality=True)
model.fit(df);

We've instantiated the model, now we need to build some future dates to forecast into.

In [ ]:
future = model.make_future_dataframe(periods=24, freq = 'm')
future.tail()

To forecast this future data, we need to run it through Prophet's model.

In [ ]:
forecast = model.predict(future)

The resulting forecast dataframe contains quite a bit of data, but we really only care about a few columns.  First, let's look at the full dataframe:

In [ ]:
forecast.tail()

We really only want to look at yhat, yhat_lower and yhat_upper, so we can do that with:

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

### Plotting Prophet results

Prophet has a plotting mechanism called ```plot```.  This plot functionality draws the original data (black dots), the model (blue line) and the error of the forecast (shaded blue area).

In [ ]:
model.plot(forecast);

Personally, I'm not a fan of this visualization but I'm not going to build my own...you can see how I do that here:
https://github.com/urgedata/pythondata/blob/master/fbprophet/fbprophet_part_one.ipynb.

Additionally, prophet let's us take a at the components of our model, including the holidays. This component plot is an important plot as it lets you see the components of your model including the trend and seasonality (identified in the ```yearly``` pane).

In [ ]:
model.plot_components(forecast);

Now that we have our model, let's take a look at how it compares to our actual values using a few different metrics - R-Squared and Mean Squared Error (MSE).

To do this, we need to build a combined dataframe with yhat from the forecasts and the original 'y' values from the data.

In [ ]:
metric_df = forecast.set_index('ds')[['yhat']].join(df.set_index('ds').y).reset_index()

In [ ]:
metric_df.tail()

You can see from the above, that the last part of the dataframe has "NaN" for 'y'...that's fine because we are only concerend about checking the forecast values versus the actual values so we can drop these "NaN" values. 

In [ ]:
metric_df.dropna(inplace=True)

In [ ]:
metric_df.tail()

Now let's take a look at our R-Squared value

In [ ]:
r2_score(metric_df.y, metric_df.yhat)

An r-squared value of 0.99 is amazing (and probably too good to be true, which tells me this data is most likely overfit).

In [ ]:
mean_squared_error(metric_df.y, metric_df.yhat)

That's a large MSE value...and confirms my suspicion that this data is overfit and won't likely hold up well into the future. Remember...for MSE, closer to zero is better.

Now...let's see what the Mean Absolute Error (MAE) looks like.

In [ ]:
mean_absolute_error(metric_df.y, metric_df.yhat)

Not good. Not good at all. BUT...the purpose of this particular post is to show some usage of R-Squared, MAE and MSE's as metrics and I think we've done that.  

I can tell you from experience that part of the problem with this particular data is that its monthly and there aren't that many data points to start with (only 72 data points...not ideal for modeling).

# Another approach for metrics

While writing this post, I came across ML Metrics (https://github.com/benhamner/Metrics), which provides 17 metrics for Python (python version here --> https://github.com/benhamner/Metrics/tree/master/Python). 

Let's give it a go and see what these metrics show us.

In [ ]:
import ml_metrics as metrics

In [ ]:
metrics.mae(metric_df.y, metric_df.yhat)

Same value for MAE as before...which is a good sign for this new metrics library. Let's take a look at a few more.

Here's the Absolute Error (pointwise...shows the error of each date's predicted value vs actual value)

In [ ]:
metrics.ae(metric_df.y, metric_df.yhat)

Let's look at Root Mean Square Error

In [ ]:
metrics.rmse(metric_df.y, metric_df.yhat)

This new metrics library looks interesting...I'll keep in in my toolbox for future use. 

Hopefully this was useful for you to see how to look at some useful metrics for analyzing how good/bad your models are.  These approaches can be used for most other modeling techniques other than facebook prophet.